In [2]:
import io
import subprocess
import re
import pickle

# TPTP Formatting
fof(name,formula_type,statement)
- name is the name of the formula
- formula_type is either axiom or conjecture, for our purposes.
- Axioms are facts
- Conjectures are what we want to prove/disprove


Extra TPTP notes:
- ! is the universal quantifier
- ? is the existential quantifier
- variables are denoted with capital letters
- func(C) is a function with variable C
- => is the implication operator
- & is AND
- | is OR
- ~ is NEGATION
- <=> is EQUIVALENCE
- = is EQUALITY
- != is INEQUALITY

In [3]:
class Axiom:
    def __init__(self,name,statement):
        self.name = name
        self.statement = statement
    def __str__(self):
        return "fof("+str(self.name)+",axiom,"+str(self.statement)+")."

class Conjecture:
    def __init__(self,name,statement):
        self.name = name
        self.statement = statement
    def __str__(self):
        return "fof("+str(self.name)+",conjecture,"+str(self.statement)+")."

In [4]:
theorum = str(Axiom("1","a"))+str(Axiom("2","a=>b"))+str(Axiom("3","b=>c"))+str(Conjecture("c1","c"))

file_path = "f.tptp"
eprover_path = "/home/anmarch/source/eprover/PROVER/eprover"
with io.open(file_path,'w',encoding='utf-8') as f:
    f.write(theorum)

result = subprocess.run([eprover_path, "--proof-object", str(file_path)], capture_output=True)
output = result.stdout.decode()

if result.returncode == 0:
    #proof found
    print(output)
    pass

elif result.returncode == 1:
    #proof not found
    print(output)
    pass

else:
    #something else happened
    print(result)
    raise Exception("Something unexpected occured")

# Initializing proof state
# Scanning for AC axioms
#
#cnf(i_0_1, plain, (a)).
#
#cnf(i_0_2, plain, (b)).
#
#cnf(i_0_3, plain, (c)).
#
# Proof found!
# SZS status Theorem
# SZS output start CNFRefutation
fof(2, axiom, (a=>b), file('f.tptp', 2)).
fof(c1, conjecture, c, file('f.tptp', c1)).
fof(3, axiom, (b=>c), file('f.tptp', 3)).
fof(1, axiom, a, file('f.tptp', 1)).
fof(c_0_4, plain, (~a|b), inference(fof_nnf,[status(thm)],[inference(fof_nnf,[status(thm)],[2])])).
fof(c_0_5, negated_conjecture, ~c, inference(fof_simplification,[status(thm)],[inference(assume_negation,[status(cth)],[c1])])).
fof(c_0_6, plain, (~b|c), inference(fof_nnf,[status(thm)],[inference(fof_nnf,[status(thm)],[3])])).
cnf(c_0_7, plain, (b|~a), inference(split_conjunct,[status(thm)],[c_0_4])).
cnf(c_0_8, plain, (a), inference(split_conjunct,[status(thm)],[1])).
fof(c_0_9, negated_conjecture, ~c, inference(fof_nnf,[status(thm)],[c_0_5])).
cnf(c_0_10, plain, (c|~b), inference(split_conjunct,[status(thm)],[c_0_6])).
cn

# Parse the proof output.
Todo: get rid of unnecessary info like filenames

In [5]:
parsed_result = ""
for line in output.split('\n'):
    if len(line) > 0 and line[0] == '#':
        pass
    else:
        line = line.replace(' ','')
        print(line)
        break
        

fof(2,axiom,(a=>b),file('f.tptp',2)).


# Generate expressions using:
- and &
- or |
- not ~
- implies =>
- a1,a2,a3 etc for axiom names
- c1,c2,c3 etc for conjecture names


In [ ]:
operation_list = ['&','|']
variable_list1 = ['a','b','c','d']

original = ['a','b','c']
prop_names = []

for x in original:
    prop_names.append(str(x))

ops = ['&','!']
new = dict()
new2 = list()


for name in prop_names:
    new[str(name)]=str(name)


for i in range(5):
    for var1 in original:
        for var2 in prop_names:
            for o in ops:
                new[str(var1)+str(o)+str(var2)]=str(var1)+str(o)+str(var2)
        
    for e in new:
        original.append(str(new[e]))

for element in new:
    for name in prop_names:
        result = str(new[element])+"=>"+str(name)
        new2.append(result)

print(len(new2))
with open('data.pickle','wb') as f:
    pickle.dump(new2,f)

83979


In [8]:
with open('data.pickle','rb') as f:
    data = pickle.load(f)